### PRED 420-56 Winter 2016 GrEx2 Comments and Feedback

### Duplicated Records

How you go about this depends on how you decide to define "duplicate." You need to define what fields need to match in two records for the records to be duplicates.  Maybe you decided that all fields need to match (ok for the present purposes), or that some subset of the fields need to match.  Suppose that for the airlines.dat data you think that you should consider duplicate records to be those that have the same values in in the fields "airlineID" (the OpenFlights ID), and "Name" (airline name).

Given this definition, you need to find out how many times there are two or more records having the same values in these two fields.  Assuming that you have the airlines.dat data in a DataFrame called airLinesDF, one way you could get an answer is:

This would return for you the number of duplicated (= "True") and unique ("False") records in airLinesDF, given our definition.  The answer for this example is (probably) no duplicates.  Try it out with your data to see what you get.

Whether you should have eliminated the duplicate records before completing the rest of the assignment wasn't specified in the assignment description, so it was up to you.

### Data Types in your DataFrames

The simplest way is probably this:

This would return a series with the data types of the columns in airLinesDF.

### Row Index Values

The assignment asked you to list the first ten (10) row index values of your DataFrames.  The index of a DataFrame needn't be numeric, but it will often be by default, i.e. if you don't specify otherwise.  Its values just need to be unique, like an unique database key.  To get the first 10 values in the index of airLinesDF, you could just do:

These values would be numeric or some other data type.  The object returned would be of type pandas.core.index.Int64Index.  If you got integer values, why do you think that happened?


### Defunct Airlines

The airlines.dat file has a field called "Active," (or "active") which is coded as "Y" for yes (not defunct), and "N" for defunct.  So as a first approximation you could just count the number of times that "Y" and "N" occur:

The result you get will depend on whether you removed any duplicate records.  Did you see any codes other than Y and N?

Also, note that in the GrExercise 1 notebook it was mentioned that this "Active" field isn't reliable:

_This field is not reliable: in particular, major airlines that stopped flying long ago, but have not had their IATA code reassigned (eg. Ansett/AN), will incorrectly show as "Y"._

This raises the question of how the available data could be used to determine which if any airlines coded as active are actually defunct.  How about checking to see if there are "active" airlines in airlines.dat that aren't in routes.dat?

### Flights from Nowhere

One way to define a "flight from nowhere" is as a route that doesn't originate at an airport of record.  The routes.dat file has an OpenFlights airport ID field, and also IATA and IACO airport codes.  So you can define a flight from nowhere as a route that doesn't have as its source airport an airport that's in airports.dat.

The question is, what data do we use to verify that a source airport in the routes data is an "airport of record?"  We could use OpenFlight's airport ID number in the airports data.  Or, we could use the airports' IATA/FAA or ICAO codes.  Or, we could use both the IDs and the codes.  We could test for each condition (no airport ID in the airports data, and no IATA/IACO airport code in the airports data) separately, and then decide how to use the results.

There are different ways to go about accomplishing this.  Here's one of them, using the .isin method.  Another way is to use Python sets that include airport identifiers to find routes data source airports that aren't in the airports data.  Can you think of a third way?

Note that the airlines.dat data has separate fields for IATA and IACO airport codes.  In the routes.dat data these codes can both appear in the same field, the source airport field. So we can look for what's in this field in the two fields of the airlines.dat data.  Also, here we're not considering the possibility of duplicate codes or IDs, but looking into them would be a Good Thing to do.  Lastly, a key assumption is that the airports data is correct.

Each of the above returns a pandas Series with True and False values.  Are there any routes that have neither code?

The last statement would give you something like the following, from which we can see that there are 235 routes that don't have either an IATA code or a ICAO code for their source airport:

We haven't yet looked at the OpenFlights airport ID to see whether there are any route source airports in the routes.dat data that don't have an airport ID in the airports.dat data.  We could do what we just did to find routes that don't have source ICAO or IATA codes in the airports.dat data.  Or, we can just look at the 235 routes that we know don't have airport IATA or ICAO codes in the airports.dat.  Let's do that.

Hm.  It might be a good idea to look more closely at those source airport ID values.

Our definition of a "flight from nowhere" here is one that originates from a source airport that we can't find in the airports.dat data.  Here, if a source airport in routes.dat doesn't have an IATA or ICAO code in airports.dat, and if it also doesn't have an OpenFlights airport ID in airports.dat, then we'd call a route with that airport as its origin a "flight from nowhere."

### Extra Credit

What you needed to do is to find or write some Python code for calculating distances based on lattitude and longitude. Here's some code that you can find on the Web:

In [2]:
# Using the Haversine formula for geographic Great Circle Distance
#
# As per https://en.wikipedia.org/wiki/Haversine_formula
#
# From https://opentechschool.github.io/python-data-intro/files/geo_distance.py (I think)

from math import cos,radians,sin,pow,asin,sqrt

def distance(lat1, long1, lat2, long2):
    radius = 6371 # radius of the earth in km, roughly https://en.wikipedia.org/wiki/Earth_radius

    # Lat,long are in degrees but we need radians
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    long1 = radians(long1)
    long2 = radians(long2)

    dlat = lat2-lat1
    dlon = long2-long1

    a = pow(sin(dlat/2),2) + cos(lat1)*cos(lat2)*pow(sin(dlon/2),2)
    distance = 2 * radius * asin(sqrt(a))

    return distance


This defines a function called _distance_ that accepts two longitude/lattitude value pairs, and it returns a _Great Circle_ distance between them.  This approximation to "true" geopgraphic distance takes into account the curvature of the Earth. Note the units that the distance returned is in.

The routes.dat data include source airport IDs, and the airports.dat data include longitude and lattitude for airports, including the longitude and lattitude for O'Hare Airport in Chicago, ORD, which you need in order to calculate distances.

What we want to do is to use the above function to compute distance from ORD for each airport in the airports.dat data, and then sort the airports based on distance from ORD to find the ten that are furthest from ORD.  According to what's in the airports.dat data, ORD's lattitude is 41.978603, and its longitude is -87.904842.  Using this result the airports.dat data, and the above function, we can calculate a distance from ORD for each airport in airports.dat. Let's modify the distance function to make ORD's lattitude and longitude constants, and then apply it to the lattitude and longitude columns of airportsDF, a Dataframe with the airports.dat data in it, to get a distance from ORD measure for each airport.


This version accepts a list with two elements in it, a lattitude and a longitude, and it returns a distance.  Let's apply it to the airports' lattitudes and longitudes.  Let's assume that you have a DataFrame called __destApsDF__ that includes the airport names, lattitudes, and longitudes, and that only contains data for airports that are destinations of routes originating at ORD, i.e. that have ORD as the source airport.  (You already know how do to this.)

If you can find the longitude and lattitude of your home (there are tools available online), you can use the above to determine whether the airport you named as the one closest to your home in GrExercise 1 is indeed the closest airport to you.